In [1]:
import tensorflow as tf
import numpy as np
from PIL import Image

C:\Users\Manju\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
#import MINIST dataset
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True) # y labels are oh-encoded
#supress warnings
import warnings
warnings.filterwarnings("ignore")

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
# dataset size
n_train = mnist.train.num_examples # 55,000
n_validation = mnist.validation.num_examples # 5000
n_test = mnist.test.num_examples # 10,000

In [5]:
# network architecture
n_input = 784 	# input layer (28x28 pixels)
n_hidden1 = 512 # 1st hidden layer
n_hidden2 = 256 # 2nd hidden layer
n_hidden3 = 128 # 3rd hidden layer
n_output = 10 	# output layer (0-9 digits)


In [6]:
# network parameters
learning_rate = 1e-4
n_iterations = 1000
batch_size = 128
dropout = 0.5

In [7]:
# tf placeholders
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_output])
keep_prob = tf.placeholder(tf.float32) # dropout

In [8]:
# w & b parameters
weights = {
	'w1': tf.Variable(tf.truncated_normal([n_input, n_hidden1], stddev=0.1)),
	'w2': tf.Variable(tf.truncated_normal([n_hidden1, n_hidden2], stddev=0.1)),
	'w3': tf.Variable(tf.truncated_normal([n_hidden2, n_hidden3], stddev=0.1)),
	'out': tf.Variable(tf.truncated_normal([n_hidden3, n_output], stddev=0.1)),
}
biases = {
	'b1': tf.Variable(tf.constant(0.1, shape=[n_hidden1])),
	'b2': tf.Variable(tf.constant(0.1, shape=[n_hidden2])),
	'b3': tf.Variable(tf.constant(0.1, shape=[n_hidden3])),
	'out': tf.Variable(tf.constant(0.1, shape=[n_output]))
}

In [9]:
# network layers
layer_1 = tf.add(tf.matmul(X, weights['w1']), biases['b1'])
layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
layer_3 = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
layer_drop = tf.nn.dropout(layer_3, keep_prob)
output_layer = tf.matmul(layer_3, weights['out']) + biases['out']


In [10]:
# define loss and optimiser
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=output_layer))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [11]:
# define evaluation
correct_pred = tf.equal(tf.argmax(output_layer, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


In [12]:
# initialise variables, start session
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)


In [13]:
# train on minibatches
for i in range(n_iterations):
	batch_x, batch_y = mnist.train.next_batch(batch_size)
	sess.run(train_step, feed_dict={X: batch_x, Y: batch_y, keep_prob:dropout})
	
	# print loss and accuracy (per minibatch)
	if i%100==0:
		minibatch_loss, minibatch_accuracy = sess.run([cross_entropy, accuracy], feed_dict={X: batch_x, Y: batch_y, keep_prob:1.0})
		print("Iteration", str(i), "\t| Loss =", str(minibatch_loss), "\t| Accuracy =", str(minibatch_accuracy))



Iteration 0 	| Loss = 3.5758114 	| Accuracy = 0.0703125
Iteration 100 	| Loss = 0.52361745 	| Accuracy = 0.828125
Iteration 200 	| Loss = 0.23441075 	| Accuracy = 0.9375
Iteration 300 	| Loss = 0.32749742 	| Accuracy = 0.9140625
Iteration 400 	| Loss = 0.3424691 	| Accuracy = 0.890625
Iteration 500 	| Loss = 0.23532505 	| Accuracy = 0.921875
Iteration 600 	| Loss = 0.4300428 	| Accuracy = 0.8671875
Iteration 700 	| Loss = 0.28514856 	| Accuracy = 0.9296875
Iteration 800 	| Loss = 0.22760442 	| Accuracy = 0.9140625
Iteration 900 	| Loss = 0.32301265 	| Accuracy = 0.921875


In [14]:
# accuracy on test set
test_accuracy = sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob:1.0})
print("\nAccuracy on test set:", test_accuracy)



Accuracy on test set: 0.917


In [17]:
# accuracy on own image
img = np.invert(Image.open("test_img.png").convert('L')).ravel()
prediction = sess.run(tf.argmax(output_layer,1), feed_dict={X: [img]})
print ("Prediction for test image:", np.squeeze(prediction))

Prediction for test image: 2


In [22]:
# accuracy on own image
img = np.invert(Image.open("dig00.png").convert('L')).ravel()
#img = np.reshape(img, (1,784))

prediction = sess.run(tf.argmax(output_layer,1), feed_dict={X: [img]})
print ("Prediction for test image:", np.squeeze(prediction))

Prediction for test image: 0
